In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import os

In [2]:
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
os.getcwd()

'/root/autodl-tmp'

In [4]:
# Folder Path
path = "/root/autodl-tmp/Data/aclImdb/train/neg"
# Change the directory
os.chdir(path)

train_neg = []

# Read text File
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        train_neg.append(f.read())
  

# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)

In [5]:
path = "/root/autodl-tmp/Data/aclImdb/train/pos"
# Change the directory
os.chdir(path)

train_pos = []

# Read text File
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        train_pos.append(f.read())
  

# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)

In [6]:
print(len(train_pos)) # number of positive comments
print(len(train_neg)) # number of negative comments

12500
12500


In [7]:
texts_train = train_neg + train_pos
y_train_neg = [0]*len(train_neg)
y_train_pos = [1]*len(train_pos)
y_train = np.array(y_train_neg + y_train_pos)

In [8]:
vocabulary = 10000
tokenizer = Tokenizer(num_words=vocabulary)
tokenizer.fit_on_texts(texts_train) # build dictionary

word_index = tokenizer.word_index # word dictionary
sequence_train = tokenizer.texts_to_sequences(texts_train) # text to sequence

In [9]:
# align sequences
word_num = 500 # different with logistic regression
X_train = preprocessing.sequence.pad_sequences(sequence_train, maxlen=word_num) # Pre-padding or removing values from the beginning of the sequence is the default.

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
embedding_dim = 32
state_dim = 32

model = Sequential()
model.add(Embedding(vocabulary, embedding_dim, input_length=word_num))
model.add(SimpleRNN(state_dim, return_sequences=False)) # only return last state
model.add(Dense(1, activation='sigmoid'))

model.summary()

2022-03-06 13:43:02.775556: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


2022-03-06 13:43:03.512438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22312 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b4:00.0, compute capability: 8.6


In [ ]:
epochs = 3 # avoid overfitting
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_valid, y_valid))

/root/miniconda3/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/3


2022-03-06 13:43:05.934444: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


625/625 [==============================] - 156s 246ms/step - loss: 0.5849 - acc: 0.6643 - val_loss: 0.4825 - val_acc: 0.7838
Epoch 2/3
625/625 [==============================] - 156s 250ms/step - loss: 0.3542 - acc: 0.8536 - val_loss: 0.5387 - val_acc: 0.7764
Epoch 3/3
625/625 [==============================] - 162s 259ms/step - loss: 0.2644 - acc: 0.8950 - val_loss: 0.3389 - val_acc: 0.8538
